In [1]:
import sys
sys.path.append('..')
from SCCA_new import *
import pandas as pd
import pickle
import os

In [2]:
folder = "250-250_0.7_0.8/"

In [3]:
vx = pd.read_csv('data/'+ folder+'vx.csv', header = None).values
vx = vx/norm(vx)
vy = pd.read_csv('data/'+ folder+'vy.csv', header = None).values
vy = vy/norm(vy)
lam = 0.8
p1 = len(vx)
p2 = len(vy)
d = p1 + p2
SigX = pd.read_csv('data/'+ folder+'SigX.csv', header = None).values
SigY = pd.read_csv('data/'+ folder+'SigX.csv', header = None).values
SigXY = lam * SigX.dot(vx).dot(vy.T).dot(SigY)
Sigma = np.concatenate((np.concatenate((SigX, SigXY), axis=1),
                        np.concatenate((SigXY.T, SigY), axis=1)), axis = 0)

A = np.vstack((np.hstack((np.zeros(shape = (p1, p1)), SigXY))
                    ,np.hstack((SigXY.T, np.zeros(shape = (p2, p2))))))

B = np.vstack((np.hstack((SigX, np.zeros(shape = (p1, p2))))
                    ,np.hstack((np.zeros(shape = (p2, p1)), SigY))))


In [5]:
## Use initial value from RidgeCCA
num_data = 100
truncate = 1
sigmasq = 0.5
rho_0 = 10
Niter = 10000
step = 0.1
results = np.zeros(shape = (num_data, 5))
res_folder = 'result/' + folder + 'NormInitm1_ST_%.1f_%.1f_%.1f_%f/' %(truncate, sigmasq, rho_0, step)
isExist = os.path.exists(res_folder)
Ifplot = False

if not isExist:
  
  # Create a new directory because it does not exist 
    os.makedirs(res_folder)
    print("The new directory is created!")
for ii in range(num_data):
    
    # Generate data
#     Z =  stats.multivariate_normal.rvs(np.zeros(d), Sigma, n)
    Z =  pd.read_csv('data/'+ folder+'data%i.csv' %ii, header = None).values
    init = pd.read_csv('data/'+ folder+'minit%i.csv' %ii, header = None).values[:,0]
    init = init / np.std(init)
    n = len(Z)
    
    X = Z[:, 0:p1]
    Y = Z[:,p1:]
    # center columns of X

    Sighat_X = (X-np.mean(X)).T.dot(X-np.mean(X))
    Sighat_Y = (Y-np.mean(Y)).T.dot(Y - np.mean(Y))
    Sighat_XY = (X-np.mean(X)).T.dot(Y - np.mean(Y))

    Ahat = np.vstack((np.hstack((np.zeros(shape = (p1, p1)), Sighat_XY))
                        ,np.hstack((Sighat_XY.T, np.zeros(shape = (p2, p2))))))

    Bhat = np.vstack((np.hstack((Sighat_X, np.zeros(shape = (p1, p2))))
                        ,np.hstack((np.zeros(shape = (p2, p1)), Sighat_Y)))) / (n*2)
    
    
    now1 = datetime.now()
    error, Res_theta, Res_delta, swap_atem, swap, mAcc, loggam, I, phi,aa, visits = Simu_t(Ahat, Bhat, vx[:, 0], vy[:, 0], d, n,A,B, temp = np.arange(1,0.5, -0.1)
                                                , sigmasq = sigmasq, initial=[init, np.ones(shape = d)], Niter = Niter, update = True, rho_0= rho_0, 
                                                                                           loggam = np.zeros(5) -4, truncate = truncate, step = step)
#     burnin = 700
    if Ifplot:
        fig, ax = plt.subplots(4, 1, sharey=False, tight_layout=True , figsize = (10,10))

        ax[0].plot( error[I==0,2], 'b.', alpha=0.5 )
        ax[0].set_xlabel("Iteration")
        ax[0].set_ylabel(r"$error$")
        ax[0].set_title(r"error for $v_x$ when p=%i & N=%i" % (d, n))
        ax[0].grid()

        ax[1].plot( error[:,3], 'b.', alpha=0.5 )
        ax[1].set_xlabel("Iteration")
        ax[1].set_ylabel(r"$error$")
        ax[1].set_title(r"error for $v_y$ when p=%i & N=%i" % (d, n))
        ax[1].grid()

        ax[2].plot(Res_theta[:, 0]) 
        ax[2].set_title(r'$\theta$')

        ## eigenvalue
        ax[3].plot(error[:,1], 'b.')
        ax[3].set_title(r"eigenvalues")
        ax[3].grid()

        plt.show()
    results[ii, 4] = (datetime. now() - now1).total_seconds()
    ll = Res_delta[I==0].shape[0]
    burnin = ll//4*3
    results[ii, :4] = np.mean(error[I==0,:4][burnin:], axis = 0 )
    with open(res_folder + 'error-time%i.pkl' %(ii), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([error[I==0, :], Res_theta[I==0, :], Res_delta[I==0, :]], f)

    
    print(ii, results[ii,:])

The new directory is created!
0 [2.46144815e-03 8.28488495e-01 1.74320674e-02 8.46344375e-03
 3.75120334e+02]
1 [2.44787639e-03 7.90210280e-01 2.54097821e-02 1.79929832e-02
 3.81394840e+02]
2 [2.45600598e-03 8.17248287e-01 1.54293829e-02 9.57474586e-03
 3.78738945e+02]
3 [2.40724567e-03 7.85542409e-01 4.31083887e-02 4.76879859e-02
 3.93543858e+02]
4 [2.46417722e-03 7.97182387e-01 1.05732041e-02 1.19026910e-02
 3.75528584e+02]
5 [2.44004038e-03 8.19742567e-01 2.45142143e-02 2.19571264e-02
 3.76935074e+02]
6 [2.45863800e-03 7.92819203e-01 1.11807236e-02 1.50955896e-02
 3.74921229e+02]
7 [2.39407222e-03 7.58318615e-01 6.39659240e-02 2.98279389e-02
 3.96098922e+02]
8 [2.44489861e-03 8.38097843e-01 2.34395112e-02 1.94484485e-02
 3.72636568e+02]
9 [2.45125228e-03 8.08518703e-01 1.80901322e-02 1.85022869e-02
 3.80526494e+02]
10 [2.43867601e-03 8.07109515e-01 1.60566763e-02 3.20627698e-02
 3.75861455e+02]
11 [2.46515717e-03 8.00258549e-01 7.50457127e-03 1.08269902e-02
 3.75260109e+02]
12 [2.45

## Computation time

In [4]:
### read result from ST with scaling sigmasq
num_data = 100
d = 500
truncate = 1
sigmasq = 0.5
rho_0 = 10
Niter = 3100
step = 0.1
error_ST = np.zeros(shape = (6, num_data, Niter))
time_ST = np.zeros(shape = (num_data, Niter))
iter_max = np.zeros(shape = (num_data), dtype = np.int32)
for ii in range(0,num_data):
    with open('result/'+ folder + 'NormInitm1_ST_%.1f_%.1f_%.1f_%f/error-time%i.pkl' %(truncate, sigmasq, rho_0, step, ii), 'rb') as f:  # Python 3: open(..., 'wb')
        error, Res_theta, Res_delta= pickle.load(f)
        iter_max[ii] = len(error)
        error_ST[0:2, ii, :iter_max[ii]] = error[:, 2:4].T
        error_ST[2:6, ii, :iter_max[ii]] = error[:, 5:9].T
        time_ST[ii, :iter_max[ii]] = error[:, 4]


In [5]:
limit = np.mean(error_ST[0:2, :,1800:2000], axis = (2))

In [6]:
a = limit[0,:]
a = np.mean(a[a<1])
b = limit[1,:]
b = np.mean(b[b<1])

In [26]:
## Use initial value from RidgeCCA
num_data = 100
truncate = 1
sigmasq = 0.5
rho_0 = 10
Niter = 1000
step = 0.1
results = np.zeros(shape = (num_data, 4))
# res_folder = 'result/' + folder + 'NormInitm1_ST_%.1f_%.1f_%.1f_%f/' %(truncate, sigmasq, rho_0, step)
# isExist = os.path.exists(res_folder)
Ifplot = False

# if not isExist:
  
#   # Create a new directory because it does not exist 
#     os.makedirs(res_folder)
#     print("The new directory is created!")
for ii in range(num_data):
    
    # Generate data
#     Z =  stats.multivariate_normal.rvs(np.zeros(d), Sigma, n)
    Z =  pd.read_csv('data/'+ folder+'data%i.csv' %ii, header = None).values
    init = pd.read_csv('data/'+ folder+'minit%i.csv' %ii, header = None).values[:,0]
    init = init / np.std(init)
    n = len(Z)
    
    X = Z[:, 0:p1]
    Y = Z[:,p1:]
    # center columns of X

    Sighat_X = (X-np.mean(X)).T.dot(X-np.mean(X))
    Sighat_Y = (Y-np.mean(Y)).T.dot(Y - np.mean(Y))
    Sighat_XY = (X-np.mean(X)).T.dot(Y - np.mean(Y))

    Ahat = np.vstack((np.hstack((np.zeros(shape = (p1, p1)), Sighat_XY))
                        ,np.hstack((Sighat_XY.T, np.zeros(shape = (p2, p2))))))

    Bhat = np.vstack((np.hstack((Sighat_X, np.zeros(shape = (p1, p2))))
                        ,np.hstack((np.zeros(shape = (p2, p1)), Sighat_Y)))) / (n*2)
    
    
    now1 = datetime.now()
    error, Res_theta, Res_delta, swap_atem, swap, mAcc, loggam, I, phi,aa, visits = Simu_t(Ahat, Bhat, vx[:, 0], vy[:, 0], d, n,A,B, temp = np.arange(1,0.5, -0.1)
                                                , sigmasq = sigmasq, initial=[init, np.ones(shape = d)], Niter = Niter, update = True, rho_0= rho_0, 
                                                                                           loggam = np.zeros(5) -4, truncate = truncate, step = step, tol = 0.0001, err = [a, b])
#     burnin = 700
    results[ii, 1] = (datetime. now() - now1).total_seconds()
    results[ii, 0] = len(Res_theta)
    results[ii, 2:4 ] = error[-1, 2:4]
#     with open(res_folder + 'error-time%i.pkl' %(ii), 'wb') as f:  # Python 3: open(..., 'wb')
#         pickle.dump([error[I==0, :], Res_theta[I==0, :], Res_delta[I==0, :]], f)

    
    print(ii, results[ii,:])

0 [2.05000000e+02 7.77800600e+00 8.39668619e-03 4.60517685e-02]
1 [1.18000000e+02 5.16105400e+00 2.37025559e-02 7.37246870e-04]
2 [1.54000000e+02 6.15899600e+00 1.69200638e-02 3.64449552e-02]
3 [2.45000000e+02 1.02086820e+01 2.56017369e-02 8.85702568e-03]
4 [2.59000000e+02 9.96952900e+00 4.99164262e-03 2.80380761e-02]
5 [2.34000000e+02 9.35515800e+00 3.40258531e-02 1.04699778e-02]
6 [5.03000000e+02 1.85525730e+01 5.83156603e-03 3.30011421e-02]
7 [9.99000000e+02 3.89316880e+01 6.79813685e-02 7.62172406e-02]
8 [1.32000000e+02 5.38242500e+00 1.68554566e-02 2.85171487e-02]
9 [2.65000000e+02 1.01810380e+01 2.99205477e-02 2.05929379e-02]
10 [1.09000000e+02 4.69017000e+00 4.42051161e-02 7.91933680e-03]
11 [1.09000000e+02 4.63314300e+00 3.29611494e-02 2.19362857e-03]
12 [1.84000000e+02 7.14914200e+00 3.57294529e-02 7.45445282e-03]
13 [1.43000000e+02 6.22274000e+00 2.27015793e-02 9.85713440e-03]
14 [1.97000000e+02 7.95470900e+00 3.56104590e-03 2.69448829e-02]
15 [1.93000000e+02 7.91646500e+00 2

In [27]:
np.mean(results, axis = 0)

array([3.04050000e+02, 1.20473735e+01, 1.10161245e-01, 1.07353059e-01])